# 2. Hamlet

1. 대사가 가장 많은 등장인물 top 3?
2. 가장 많이 등장하는 단어 top 30?

## 2-1. 웹에서 연극 햄릿 원문 데이터 가져와 저장하기

Requests 패키지 불러오기

In [ ]:
import requests
help(requests)

웹사이트에서 데이터 가져오기, 결과 살펴보기

In [ ]:
url = 'http://shakespeare.mit.edu/hamlet/full.html'
r = requests.get(url)
print(r)
print(type(r))
print(r.headers)
print(r.encoding)
print('---')
print(r.text[:1000])

데이터 저장하기

In [ ]:
fname = 'hamlet.txt'

In [ ]:
with open(fname, 'w') as fout:
    fout.write(r.text)

함수 형태로 정리한 코드

In [ ]:
import requests
    
def get_data(url, fname):
    r = requests.get(url)
    with open(fname, 'w') as fout:
        fout.write(r.text)
    print('"{}" is saved.'.format(fname), end=' ')
    print('The length of data is {:,}.'.format(len(r.text)))

url = 'http://shakespeare.mit.edu/hamlet/full.html'
fname = 'hamlet.txt'
get_data(url, fname)

## 2-2. 저장한 데이터 불러오기

In [ ]:
with open(fname, 'r') as fin:
    data = fin.read()
    data = data.replace('A NAME', 'A ID') #for find_all method

You may need beautifulsoup...

In [ ]:
data[:1000]

### 2-3. beautifulsoup 사용해서 데이터 파싱하기

Documentation: https://cryptosan.github.io/pythondocuments/documents/beautifulsoup4/

In [ ]:
from bs4 import BeautifulSoup as bsoup

In [ ]:
help(bsoup)

일반적인 html 문서의 구조

```
<html>
   <head>
      <title>
         Example
      </title>
   </head>

   <body>
      <h1>Hello, world</h1>
   </body>
</html>
```

이는 beautifulsoup을 사용하면 트리 형태로 파싱된다

In [ ]:
soup = bsoup(data, 'html.parser')
type(soup)

In [ ]:
soup.head

`find_all` 메서드 사용하여 파싱한 트리 탐색하기
- 시그니처: `find_all(name, attrs, recursive, text, limit, **kwargs)`

In [ ]:
lines = soup.find_all('blockquote')
len(lines)

### 2-4. 등장인물 부분만 담은 리스트, 대사만 담은 리스트 만들기

#### 데이터 특징
- 등장인물: `<A NAME=speech23><b>BERNARDO</b></a>`
- 대사: `<A NAME=1.1.1>Who's there?</A>`

#### 위에서 데이터를 불러올 때 NAME을 ID로 바꿨으니, 아래와 같은 특징을 가진다
- 등장인물: `<A ID=speech23><b>BERNARDO</b></a>`
- 대사: `<A ID=1.1.1>Who's there?</A>`

정규표현식 모듈을 사용한 패턴 매칭 예시

In [ ]:
import re
p_char = re.compile('speech\d*')
p_line = re.compile('\d*\.\d*\.\d*')
print(p_char.match('speech23'))
print(p_line.match('3.4.127'))

In [ ]:
print(soup.find_all('a', id='1.1.1'))

In [ ]:
char_list = soup.find_all(id=p_char) #등장인물 태그 리스트
line_list = soup.find_all(id=p_line) #대사 태그 리스트
print(len(char_list), len(line_list))

In [ ]:
char_list[0], line_list[0]

In [ ]:
type(char_list[0]), type(line_list[0])

등장인물, 대사 태그에서 텍스트 부분만 남기기

In [ ]:
char_list = [c.text for c in char_list]
line_list = [l.text for l in line_list]
print(char_list[:10])
print(line_list[:10])

### 2-5. 개수 세어보기

등장인물 개수 세어보기

In [ ]:
dic = {}
for c in char_list:
    if c in dic.keys():
        dic[c] += 1
    else:
        dic[c] = 1

In [ ]:
count_pairs = list(dic.items())
count_pairs.sort(key=lambda x: x[1], reverse=True)
for char, count in count_pairs:
    print('{:20}: {}'.format(char, count))

대사에 등장하는 단어 개수를 세어보기 전에 전처리를 조금 해보자

In [ ]:
def preprocessor(line):
    import re
    for x in '.:,?!':
        line = line.replace(x, ' ')
    p = re.compile(' +')
    line = p.sub(' ', line)
    line = line.strip()
    line = line.lower()
    return line

lines = [preprocessor(l) for l in line_list]

In [ ]:
dic = {}
for l in lines:
    l = l.split(' ')
    for w in l:
        if w in dic.keys():
            dic[w] += 1
        else:
            dic[w] = 1

In [ ]:
count_pairs = list(dic.items())
count_pairs.sort(key=lambda x: x[1], reverse=True)
for char, count in count_pairs:
    if count > 30:
        print('{:6}: {}'.format(char, count))

참고: 파이썬 표준 라이브러리를 보다 쉽게 개수를 셀 수 있다

In [ ]:
from collections import Counter

char_counter = Counter(char_list).items()
char_counter